# 도란보리 Week 1 Assignment

도란보리 소모임 1주차 과제입니다. 과제 구성은 다음과 같습니다.

- librosa 라이브러리 사용법 및 개념 복습
- torchaudio 라이브러리로의 확장

---

## 과제 1: DTMF 신호 분석 및 디코딩

### 개요

이번 과제는 음성 신호 처리의 응용 사례인 **DTMF(Dual-Tone Multi-Frequency)** 신호를 이해하고 분석하는 것을 목표로 합니다. 지난 주차에 학습한 **푸리에 변환**의 원리를 실제 데이터에 적용하여, 특정 주파수 성분을 분석하고 유의미한 정보를 추출하는 과정을 실습하게 됩니다.

### 이론적 배경: DTMF

아날로그 전화기가 어떻게 입력한 번호로 전화를 거는지 아시나요?? 그 속에는 아날로그 전화기 Encoder를 통한 여러 주파수 혼합음 생성과 푸리에 변환을 기본으로 한 전화국의 Decoder 기반으로 다이얼링이 이루어 진답니다! 이 기술을 간단히 알아보고 분석해 볼까요?

물론 요즘 스마트폰과 같은 전화기는 전화번호를 입력한 뒤 전화를 걸면 어느 번호를 입력했는지 정보가 담겨있는 디지털 패킷을 생성하여 전송하는 SIP 방식을 사용합니다. 하지만 과거 아날로그 전화기 시절에는 어떻게 사용자가 입력한 전화번호를 인식하고 그 번호로 전화를 걸 수 있었을까요?? 이는 바로 DTMF 기술이 있었기에 가능했습니다!

다들 옛날에 집전화 아날로그 전화기를 통해 전화를 걸어본 경험이 있을 것이라 생각하는데요!(설마 없나..^^) 번호를 누르면 삑 하는 소리가 났던 것을 기억하실 수 있을 겁니다! 이 소리는 어떤 숫자의 버튼을 누르는지에 따라 다릅니다(이를 다이얼톤이라고 합니다). 

전화기는 이러한 음성 신호를 생성해내고, 기지국의 디코더는 이러한 음성 신호의 주파수를 분석하여 어떤 숫자를 입력했는지 파악하는 것이죠! 

근데 이렇게 단순하게 다이얼톤을 구성하면 문제가 생깁니다. 만약 우리 목소리와 비슷한 진동수의 소리를 다이얼톤으로 사용한다면?? 혹은 일상에서 흔히 들리는 소리와 겹치는 진동수의 소리를 다이얼톤으로 사용한다면?? 버튼음이 아니라 잡음으로 인해 번호가 잘 못 눌릴 위험성이 있었습니다! 따라서 공학자들은 일상생활에서의 소리와 다이얼톤을 명확히 구분하기 위해서 다이얼톤은 서로 다른 두 진동수의 음을 섞어 만들었습니다!

<img src="cf1.png" width="400" height="500"/>
<img src="cf2.png" width="400" height="500"/>
<img src="cf3.png" width="400" height="500"/>

위 이미지를 참고해 주세요! 예를 들어 우리가 수화기를 들고 1을 누르면, 전화기는 697Hz, 1209Hz에 해당하는 두 사인파를 생성하고 두 사인파의 합을 통해 마지막과 같은 신호를 생성해 냅니다!

기지국의 디코더는 해당 신호를 푸리에 변환하여 "아! 이 신호에는 697Hz, 120Hz"가 혼합되어 있으니 1을 누른 거구나"로 인식하는 것이죠!

### 실습 1: DTMF 신호 생성

그래서 우리의 첫 번째 과제는 사인파를 만들어내는 wave, struct 라이브러리를 이용하여 여러 숫자에 대응되는 다이얼톤을 만들어보고, 최종적으로는 제 전화번호를 누른 다이얼톤을 들어보고 해당 음성 파일을 librosa를 통해 스펙트로그램을 그려봄으로써

**제 전화번호를 알아맞히는 것입니다 ㅎㅎ**

우선 다이얼톤을 만들어 볼까요?? 다이얼톤을 만들어내는 파이썬 코드를 작성해 두었습니다. 맨 밑 seq=""의 문자열 속 내용을 수정하면 해당 숫자에 해당하는 다이얼톤을 생성해 냅니다.

In [1]:
import numpy as np
import wave
import struct

# 각 키에 해당하는 DTMF 주파수 (저역, 고역)
DTMF = {
    '1': (697, 1209), '2': (697, 1336), '3': (697, 1477), 'A': (697, 1633),
    '4': (770, 1209), '5': (770, 1336), '6': (770, 1477), 'B': (770, 1633),
    '7': (852, 1209), '8': (852, 1336), '9': (852, 1477), 'C': (852, 1633),
    '*': (941, 1209), '0': (941, 1336), '#': (941, 1477), 'D': (941, 1633),
}

def dtmf_tone(digit, duration=0.2, sr=8000, ramp_ms=5):
    """단일 숫자/문자에 대한 DTMF 톤 생성"""
    f_low, f_high = DTMF[digit]
    t = np.arange(int(duration * sr)) / sr
    x = 0.5 * (0.707*np.sin(2*np.pi*f_low*t) + 0.707*np.sin(2*np.pi*f_high*t))
    ramp_len = int(sr * ramp_ms / 1000.0)
    if ramp_len > 0:
        ramp = 0.5 * (1 - np.cos(np.linspace(0, np.pi, ramp_len)))
        x[:ramp_len] *= ramp
        x[-ramp_len:] *= ramp[::-1]
    return x

def sequence_to_dtmf(seq, tone_ms=200, gap_ms=70, sr=8000):
    """전체 숫자열 시퀀스에 대해 톤과 무음을 이어붙여 신호 생성"""
    tone_dur = tone_ms / 1000.0
    gap = np.zeros(int(gap_ms / 1000.0 * sr))
    pieces = []
    for ch in seq:
        if ch == ' ':
            # 공백은 긴 휴지부로 처리(예: 0.3초)
            pieces.append(np.zeros(int(0.3 * sr)))
            continue
        pieces.append(dtmf_tone(ch, duration=tone_dur, sr=sr, ramp_ms=5))
        pieces.append(gap)
    x = np.concatenate(pieces) if pieces else np.zeros(1)
    # 16-bit 범위로 노멀라이즈 (여유 5%)
    x = x / (np.max(np.abs(x)) + 1e-9) * 0.95
    return x

def write_wav(path, data, sr=8000):
    """모노 16-bit PCM WAV 저장"""
    with wave.open(path, 'wb') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sr)
        pcm = (data * 32767).astype(np.int16)
        wf.writeframes(struct.pack('<' + 'h'*len(pcm), *pcm))

if __name__ == "__main__":
    seq = "12345"
    sr = 8000  # 전화 음성 표준 샘플링레이트
    audio = sequence_to_dtmf(seq, tone_ms=200, gap_ms=70, sr=sr)
    name = "dtmf_" + seq + ".wav"
    write_wav(name, audio, sr)
    print(f"Saved: {name}")

Saved: dtmf_12345.wav


어때요 신기하죠?? 해당 소리만으로 전화를 걸 수 있습니다. 실제 아날로그 전화기의 수화기를 든 뒤, 버튼을 누르지 않고도 수화기의 마이크에 대고 해당 사운드를 재생시키면 전화가 걸릴 거예요!

### 실습 2: 스펙트로그램 분석

이번에는 `librosa` 라이브러리를 사용해서 생성한 wav 파일의 스펙트로그램을 그려주는 파이썬 코드를 작성해 보세요! 스펙트로그램을 통해 각 숫자 톤에 해당하는 두 개의 주파수 성분히 명확히 나타나는지 확인해보세요.

#### 과제

`librosa.load`와 `librosa.stft`를 사용하여 스펙트로그램을 시각화하는 코드를 직접 작성해 보세요. 팁: DTMF에서 사용하는 주파수 대역(약 600Hz ~ 1700Hz)이 잘 보이도록 y축의 범위를 적절히 조절(`ylim`)하면 더욱 명확한 분석이 가능합니다.

[(스펙트로그램 예시)](https://github.com/DohyunKim-UOS/ttsstudy/blob/main/Week2_script_example/spectrogram.py)

### 최종 과제: 전화번호 디코딩

번에는 제 번호 DTMF 파일을 드릴테니 다운받아서, 스펙트로그램을 그려보고 한 번 제 번호를 맞혀보세요!

In [2]:
# 과제용 WAV 파일 다운로드
! wget https://github.com/DohyunKim-UOS/ttsstudy/raw/main/my_number.wav -O my_number.wav

--2025-10-29 21:40:41--  https://github.com/DohyunKim-UOS/ttsstudy/raw/main/my_number.wav
Resolving github.com (github.com)... 20.200.245.247
Connecting to github.com (github.com)|20.200.245.247|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DohyunKim-UOS/ttsstudy/main/my_number.wav [following]
--2025-10-29 21:40:42--  https://raw.githubusercontent.com/DohyunKim-UOS/ttsstudy/main/my_number.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47564 (46K) [audio/wav]
Saving to: ‘my_number.wav’

my_number.wav       100%[===================>]  46.45K  --.-KB/s    in 0.006s  

2025-10-29 21:40:42 (7.90 MB/s) - ‘my_number.wav’ saved [47564/47564]



#### 과제

다운로드한 my_number.wav 파일을 librosa로 로드하고 스펙트로그램을 그려, 각 시간대별로 어떤 주파수들이 활성화되는지 분석하세요. 이를 바탕으로 원본 전화번호(숫자 시퀀스)를 알아내보세요!

---

## 과제 2: `torchaudio`를 이용한 음향 신호 분석: 협화음, 불협화음, 그리고 목소리

여러분 물및실1 배우실 떼 파동, 도플러효과, 맥놀이 배웠죠?? 맥놀이를 악기 튜닝할 때 쓴다고 배운 것 같은데 과연 튜닝할 때 어떻게 맥놀이를 활용한다는 걸까요?? 한 번 알아봅시다!

### 개요

이번 과제에서는 음성 처리 분야에서 널리 사용되는 PyTorch 기반 라이브러리인 **torchaudio**의 사용법을 익힙니다. 우리는 먼저 음악의 기본 요소인 **협화음(Consonance)**과 **불협화음(Dissonance)**을 주파수 관점에서 이해하고, 이를 torchaudio를 통해 스펙트로그램과 멜 스펙트로그램으로 시각화하여 분석합니다. 최종적으로는 자신의 목소리를 직접 녹음하고 분석하여, 음성 데이터의 특징을 추출하는 과정을 실습합니다.

### 이론적 배경: 협화음과 불협화음

진동수가 클수록 음이 높을까요 아니면 진동수가 작을수록 음이 높을까요?? 다들 일반물리학 배우셨으니 잘 아실 것이라 생각합니다!

당연히 진동수가 클수록, 파장이 짧을수록, 1초에 더 많이 진동할수록 음은 높아지죠!

기타의 5번째 현 정튜님 음은 A (440Hz)입니다! 기타 튜너 없이 맥놀이를 이용해 튜닝을 진행하려고 할 때에도 440Hz에 대당하는 A음 정도는 귀로 듣고 맞출 수 있어야 합니다!

화성학적으로 한 옥타브가 높아지면 진동수는 두 배가 됩니다. 예를 들어 기본 A가 440Hz라면, 높은 A는 880Hz가 됩니다.

기타리스트들은 5번째 현을 A 440Hz에 맞추고 나머지 현들을 해당 현의 맥놀이를 이용해 튜닝을 진행합니다.

* 기준음을 A(440Hz)로 잡는 이유는, A음이 440Hz로 가장 듣기 편하고, 일의자리가 깔끔하게 0으로 나눠떨어지기 때문입니다!

기타의 5번 현은 A이고, 4번 현은 D로 맞춰야 합니다!

5번 현은 A에 맞춰두었으니 5번 현의 5번 프렛을 누르면 D음이 나게 됩니다! 그러면 이 소리를 듣고 4번 현을 이 음과 똑같이 맞추면 4번 현의 음이 맞게 되겠죠!! 이때 음의 일치 여부를 맥놀이를 통해 확인합니다 ㅎㅎ

만약 두 현에서 나는 음이 정확히 일치한다면 두 음의 위상이 같기 때문에 자연스러운 하나의 사인파 음이 들리게 됩니다.

하지만 미세하게 틀어져서 서로 다른 두 음이 동시에 들린다면??

두 음의 합성 과정에서 불규칙한 파동이 생성되어 약간의 울림이 들리게 됩니다..ㅠㅠ 이것이 **맥놀이** 현상이며 기타리스트들은 이 미묘한 불협을 느끼며 튜닝을 진행합니다.

<img src="cf4.png" width="400" height="500"/>

음악에 관심 많으신 분들이라면 코드(Chord)라는 말을 많이 들어보셨을 거예요! Chord는 협화음으로 이루어진 음의 구성이라고 생각하시면 됩니다. 

협화음, 불협화음이라는 말 많이 들어보셨죠??

피아노든 Garage Band든 있으시면 키셔서 도랑 미를 동시에 눌러보세요! 듣기 편할 겁니다.

반대로 도랑 레를 동시에 누른다면?? 무엇인가 불편할 거예요ㅠㅠ 왜그럴까요??

바로 주파수의 관계에 있습니다! 두 주파수의 관계가 단순 정수비 (2:1, 3:2, 5:4 등)라면, 두 파장의 합이 짧은 간격을 두고 주기적으로 반복되게 됩니다! DFT 영상 보시면 알 수 있을 거예요

하지만 복잡한 정수비의 두 음을 연주하게 되면, 단순 정수비로 맞아 떨어지지 않아 주기가 매우 길어져 우리 귀가 듣기 불편해지게 됩니다.

### 실습 1: 기본 사인파 생성

분석에 사용할 기본 음원을 numpy를 이용해 생성합니다. '도(C4)' 음을 기준으로, 협화음을 만드는 '솔(G4)'과 불협화음을 만드는 '도#(C#4)'에 해당하는 사인파를 생성해 봅시다.

도 (C4): 261.63 Hz

솔 (G4): 392.00 Hz (C4와 약 3:2 비율)

도# (C#4): 277.18 Hz (C4와 복잡한 비율)

In [ ]:
import torch
import torchaudio

SR = 16000  # Sample Rate
DURATION = 2  # 2 seconds

def generate_sine_wave(freq, duration, sr):
    t = np.linspace(0, duration, int(sr * duration), endpoint=False)
    sine_wave = np.sin(2 * np.pi * freq * t)
    return sine_wave

# 기본음 생성
c4_wave = generate_sine_wave(261.63, DURATION, SR)
g4_wave = generate_sine_wave(392.00, DURATION, SR)
c_sharp_4_wave = generate_sine_wave(277.18, DURATION, SR)

### 실습 2: 현화음 및 불협화음 신호 생성 및 Tensor 변환

생성한 사인파를 더하여 협화음(도+솔)과 불협화음(도+도#) 신호를 만듭니다. 그 후, torchaudio에서 처리할 수 있도록 NumPy 배열을 PyTorch Tensor 객체로 변환합니다. 이는 torchaudio 사용의 가장 기본적인 첫 단계입니다.

In [ ]:
# 협화음 (C4 + G4)
consonance_wave = c4_wave + g4_wave
# 불협화음 (C4 + C#4)
dissonance_wave = c4_wave + c_sharp_4_wave

# PyTorch Tensor로 변환 (torchaudio는 tensor를 입력으로 받습니다)
consonance_tensor = torch.from_numpy(consonance_wave).float().unsqueeze(0)
dissonance_tensor = torch.from_numpy(dissonance_wave).float().unsqueeze(0)

### 실습 3: `torchaudio`를 이용한 스펙트로그램 분석

`torchaudio.transforms.Spectrogram`을 사용하여 두 신호의 스펙트로그램을 생성하고 시각화해 봅시다.

과제: 아래 코드를 완성하여 두 스펙트로그램을 그리고, 두 이미지의 차이점을 관찰하세요. 협화음의 스펙트로그램에서 나타나는 주파수 성분들의 규칙성과 불협화음에서의 차이를 서술해 보세요.

In [ ]:
# Spectrogram Transform 정의
spectrogram_transform = torchaudio.transforms.Spectrogram(n_fft=1024)

# Spectrogram 생성
spec_consonance = spectrogram_transform(consonance_tensor)
spec_dissonance = spectrogram_transform(dissonance_tensor)

# 시각화 코드 (직접 작성해보세요)
# plt.figure() ... plt.imshow(...) ...

### 실습 4: 멜 스펙트로그램 변환

멜 스펙트로그램은 인간의 청각 인지 특성을 반영하여 주파수 축을 변환한 것으로, 특히 음성 인식 및 합성(TTS) 분야에서 핵심적인 피처(feature)로 사용됩니다. `torchaudio.transforms.MelSpectrogram`을 사용하여 멜 스펙트로그램을 생성하고, 일반 스펙트로그램과의 차이점을 비교해 보세요.

과제: 협화음 신호에 대해 멜 스펙트로그램을 생성하고, 이전 단계의 일반 스펙트로그램과 주파수 축(y-axis)이 어떻게 다른지 비교 분석해 보세요.

In [ ]:
# Mel Spectrogram Transform 정의
mel_spectrogram_transform = torchaudio.transforms.MelSpectrogram(sample_rate=SR, n_fft=1024, n_mels=80)

# Mel Spectrogram 생성
mel_spec_consonance = mel_spectrogram_transform(consonance_tensor)

# 시각화 및 비교 분석 (직접 작성해보세요)

### 최종 과제: 본인 음성 녹음 및 분석

이제 학습한 `torchaudio`의 기능들을 실제 음성 데이터에 적용해 봅시다. 아래 코드를 사용하여 자신의 목소리를 직접 녹음하고, 해당 음성 파일의 스펙트로그램과 멜 스펙트로그램을 그려보세요.

지시사항:

1. 아래 셀을 실행하여 5초간 자신의 목소리를 녹음하세요. (예: "안녕하세요, 인공지능학과 안재현입니다.")

2. 녹음된 `my_voice.wav` 파일을 `torchaudio.load` 함수로 불러오세요.

3. 불러온 음성 데이터 텐서를 이용해 스펙트로그램과 멜 스펙트로그램을 각각 생성하고 시각화하세요.

4. 자신의 목소리에 어떤 주파수 성분들이 주로 분포하는지 분석해 보세요.

In [ ]:
# Colab/Jupyter 환경에서 음성을 녹음하는 스크립트
from IPython.display import Audio, display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import io
import scipy.io.wavfile as wavfile

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record_audio(filename="my_voice.wav", duration=5):
    print(f"Recording for {duration} seconds...")
    display(Javascript(RECORD))
    s = eval_js(f'record({duration*1000})')
    print("Recording finished.")
    b = b64decode(s.split(',')[1])
    wav_io = io.BytesIO(b)
    rate, data = wavfile.read(wav_io)
    # 16-bit PCM to float
    data = data.astype(np.float32) / 32767.0
    wavfile.write(filename, rate, data)
    print(f"Saved to {filename}")
    
record_audio() # 5초간 녹음 시작